# 第5章: 係り受け解析

- CaboCha
https://taku910.github.io/cabocha/

```
brew install cabocha
```
で入った

In [1]:
!cabocha -v

cabocha of 0.69


In [2]:
!cat neko.txt | cabocha -f1 > neko.txt.cabocha

In [3]:
!head -n20 neko.txt.cabocha

* 0 -1D 0/0 0.000000
一	名詞,数,*,*,*,*,一,イチ,イチ
EOS
EOS
* 0 2D 0/0 -0.764522
　	記号,空白,*,*,*,*,　,　,　
* 1 2D 0/1 -0.764522
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 2 -1D 0/2 0.000000
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
EOS
* 0 2D 0/1 -1.911675
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 1 2D 0/0 -1.911675
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ


## 40. 係り受け解析結果の読み込み（形態素）

In [4]:
class Morph:
    def __init__(self,mor):
        self.parse(mor)

    def __str__(self):
        return self.mor
        
    def parse(self,mor):
        self.mor = mor
        splited = [mor.split('\t')[0]] + mor.split('\t')[1].split(',')
        self.surface=splited[0]
        self.base=splited[7]
        self.pos=splited[1]
        self.pos1=splited[2]     


with open('neko.txt.cabocha') as f:
    all_text = []
    sentence_list = []
    for line in f:
        if line=='EOS\n':
            if len(sentence_list)>0:
                all_text.append(sentence_list[:])
            sentence_list = []
        elif line[0]=='*':
            pass
        else:
            sentence_list.append(Morph(line))


In [5]:
for i in range(3):
    for mor in all_text[i]:
        print(mor,end='')
    print()

一	名詞,数,*,*,*,*,一,イチ,イチ

　	記号,空白,*,*,*,*,　,　,　
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。

名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ
無い	形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ
。	記号,句点,*,*,*,*,。,。,。



## 41. 係り受け解析結果の読み込み（文節・係り受け）

In [6]:
class Chunk:
    def __init__(self,ctxt):
        self.morphs = []
        self.srcs = []
        self.dst = -1
        self.parse(ctxt)
    
    def __str__(self):
        s = ""
        for m in self.morphs:
            s += m.surface
        return s
    
    def text(self):
        s = self.ctxt
        for m in self.morphs:
            s += m.__str__()
        return s        
       
    def parse(self,ctxt):
        self.ctxt = ctxt
        self.idx = int(ctxt.split()[1])
        self.dst = int(ctxt.split()[2][:-1])


In [7]:
all_text = []
sentence_list = []
srcs = {}
with open('neko.txt.cabocha') as f:
    for line in f:
        if line[0]=='*':
            c = Chunk(line)
            sentence_list.append(c)
            if c.dst in srcs.keys():
                srcs[c.dst].append(c.idx)
            else:
                srcs[c.dst] = [c.idx]
            if c.idx in srcs:
                c.srcs = srcs[c.idx][:]
        elif line=='EOS\n':
            srcs = {}
            if len(sentence_list)>0:
                all_text.append(sentence_list[:])
            sentence_list = []
        else:
            c.morphs.append(Morph(line))


In [8]:
for chunk in all_text[4]:
    print("{} {} -> {}".format(chunk.idx,chunk,chunk.dst))

0 何でも -> 1
1 薄暗い -> 3
2 じめじめした -> 3
3 所で -> 5
4 ニャーニャー -> 5
5 泣いて -> 7
6 いた事だけは -> 7
7 記憶している。 -> -1


## 42. 係り元と係り先の文節の表示

In [9]:
import re
for sentence in all_text[0:4]: 
    for chunk in sentence:
        if chunk.dst == -1: continue
        print(re.sub('[、。　]','',"{}\t{}".format(chunk,sentence[chunk.dst])))

	猫である
吾輩は	猫である
名前は	無い
まだ	無い
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

In [10]:
for sentence in all_text[0:10]:
    for chunk in sentence:
        if chunk.dst == -1: continue
        cfrom = chunk
        cto   = sentence[chunk.dst]
        ffrom = False
        fto   = False
        for mor in cfrom.morphs:
            if mor.pos=='名詞': ffrom = True
        for mor in cto.morphs:
            if mor.pos=='動詞': fto = True
        if ffrom and fto:
            print(re.sub('[、。　]','',"{}\t{}".format(cfrom,cto)))

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて
掌に	載せられて
スーと	持ち上げられた
時	フワフワした
感じが	あったばかりである


## 44. 係り受け木の可視化

graphviz : https://graphviz.gitlab.io/documentation/

pydot : https://github.com/erocarrera/pydot

```
brew install graphviz
pip install pydot
```

In [11]:
import pydot

s = 'digraph graphname{'
sentence = all_text[7]
for chunk in sentence:
    if chunk.dst == -1: continue
    s += re.sub('[、。　]','',"{} -> {} \n ".format(chunk,sentence[chunk.dst]))
s += '}'

In [12]:
g = pydot.graph_from_dot_data(s)
g[0].write_png('out.png')

![title](out.png)

## 45. 動詞の格パターンの抽出

In [13]:
def left_verb(chunk):
    for mor in chunk.morphs:
        if mor.pos == '動詞':
            return mor
    return None

def jlist(sentence,srcs):
    chunks = [sentence[src] for src in srcs]
    j = [mor.base for chunk in chunks for mor in chunk.morphs if mor.pos == '助詞']
    return j

with open('kakupattern.txt','w') as f:
    for sentence in all_text:
        for chunk in sentence:
            v = left_verb(chunk)
            j = jlist(sentence,chunk.srcs)
            if v is not None and len(j)>0:
                print(v.base,end='\t',file=f)
                print(' '.join(j),file=f)

#確認
!head kakupattern.txt

生れる	で
つく	か が
泣く	で
する	て だけ は
始める	で
見る	は を
聞く	で
捕える	を
煮る	て
食う	て


### コーパス中で頻出する述語と格パターンの組み合わせ

In [14]:
!cat kakupattern.txt | sort | uniq -c | sort -nr | head

 565 云う	と
 442 する	を
 249 思う	と
 199 ある	が
 189 なる	に
 174 する	に
 173 見る	て
 127 する	と
 117 する	が
  94 見る	を
sort: Broken pipe


### 「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [15]:
!cat kakupattern.txt | sort | uniq -c | sort -nr | grep -E "\s(する|見る|与える)\s" | head

 442 する	を
 174 する	に
 173 見る	て
 127 する	と
 117 する	が
  94 見る	を
  84 する	て を
  59 する	は
  58 する	を に
  58 する	て


## 46. 動詞の格フレーム情報の抽出

In [16]:
def jlist2(sentence,srcs):
    chunks = [sentence[src] for src in srcs]
    return ([mor.base for chunk in chunks for mor in chunk.morphs if mor.pos == '助詞'],
            [chunk for chunk in chunks for mor in chunk.morphs if mor.pos == '助詞'])

for sentence in all_text[0:8]:
    for chunk in sentence:
        v = left_verb(chunk)
        j,c = jlist2(sentence,chunk.srcs)
        if v is not None and len(j)>0:
            print(v.base,end='\t')
            print(' '.join(j),end='\t')
            print(' '.join([str(i) for i in c]))



生れる	で	　どこで
つく	か が	生れたか 見当が
泣く	で	所で
する	て だけ は	泣いて いた事だけは いた事だけは
始める	で	ここで
見る	は を	吾輩は ものを
聞く	で	あとで
捕える	を	我々を
煮る	て	捕えて
食う	て	煮て


## 47. 機能動詞構文のマイニング

In [17]:
def has_wo(chunk):
    return 'を' in [mor.base for mor in chunk.morphs]

def has_sahen(chunk):
    return 'サ変接続' in [mor.pos1 for mor in chunk.morphs]

def has_joshi(chunk):
    return '助詞' in [mor.pos for mor in chunk.morphs]

with open('kinoudoushi.txt','w') as f:
    for sentence in all_text:
        for chunk in sentence:
            v_mor = left_verb(chunk)
            if v_mor is None: continue
            s_chunks = [c for c in sentence if c.idx in chunk.srcs and has_joshi(c)]
            if len(s_chunks) < 2: continue

            for s_c in s_chunks:
                if has_wo(s_c) and has_sahen(s_c):
                    print(s_c,end='',file=f)
                    print(v_mor.base,end='\t',file=f)
                    s_chunks.remove(s_c)
                    print(' '.join([m.base for c in s_chunks for m in c.morphs if m.pos == '助詞']),
                          end='\t',file=f)
                    print(' '.join(re.sub('、','',str(c)) for c in s_chunks),file=f)
                    break

!head kinoudoushi.txt

決心をする	と	こうと
返報をする	んで	偸んで
昼寝をする	が	彼が
迫害を加える	て	追い廻して
家族的生活をする	が を	我等猫族が 愛を
投書をする	て へ	やって ほととぎすへ
話をする	に	時に
昼寝をする	て	出て
失笑するのを禁じる	は	吾輩は
欠伸をする	から て て	なったから して 押し出して


### コーパス中で頻出する述語（サ変接続名詞+を+動詞）

In [18]:
!cat kinoudoushi.txt | cut -f1 | sort | uniq -c | sort -nr | head

  25 返事をする
  19 挨拶をする
  11 話をする
   8 質問をする
   7 喧嘩をする
   6 真似をする
   5 質問をかける
   5 相談をする
   5 昼寝をする
   4 御辞儀をする


### コーパス中で頻出する述語と助詞パターン

In [19]:
!cat kinoudoushi.txt | cut -f1-2 | sort | uniq -c | sort -nr | head

   4 挨拶をする	から
   4 返事をする	と
   3 挨拶をする	と
   3 喧嘩をする	と
   2 同情を表する	と は て
   2 質問をかける	と は
   2 休養を要する	は
   2 返事をする	は と
   2 議論をする	て
   2 講義をする	で


## 48. 名詞から根へのパスの抽出